# Multimodal RAG with LangChain

This notebook shows how to perform RAG on the table, chart, and text extraction results of NV-Ingest's pdf extraction tools using LangChain

**Note:** In order to run this notebook, you'll need to have the NV-Ingest microservice running along with all of the other included microservices. To do this, make sure all of the services are uncommented in the file: [docker-compose.yaml](https://github.com/NVIDIA/nv-ingest/blob/main/docker-compose.yaml) and follow the [quickstart guide](https://github.com/NVIDIA/nv-ingest?tab=readme-ov-file#quickstart) to start everything up. You'll also need to have the NV-Ingest python client installed as demonstrated [here](https://github.com/NVIDIA/nv-ingest?tab=readme-ov-file#step-2-installing-python-dependencies).

To start, make sure LangChain and pymilvus are installed and up to date

In [ ]:
pip install -qU langchain langchain_community langchain-nvidia-ai-endpoints langchain_milvus pymilvus

Note: you may need to restart the kernel to use updated packages.


Then, we'll use NV-Ingest's Ingestor interface to extract the tables and charts from a test pdf, embed them, and upload them to our Milvus vector database (VDB)

In [2]:
from nv_ingest_client.client import Ingestor

ingestor = (
    Ingestor(message_client_hostname="localhost")
    .files("../data/multimodal_test.pdf")
    .extract(
        extract_text=False,
        extract_tables=True,
        extract_images=False,
    ).embed(
        text=False,
        tables=True,
    ).vdb_upload()
)

results = ingestor.ingest()

ERROR: Could not find a version that satisfies the requirement nv-ingest (from versions: none)
ERROR: No matching distribution found for nv-ingest
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'nv_ingest_client'

Now, the text, table, and chart content is extracted and stored in the Milvus VDB along with the embeddings. Next we'll connect LangChain to Milvus and create a vector store so that we can query our extraction results

In [4]:
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_milvus import Milvus

embedding_function = NVIDIAEmbeddings(base_url="http://localhost:8012/v1")

vectorstore = Milvus(
    embedding_function=embedding_function,
    collection_name="nv_ingest_collection",
    primary_field = "pk",
    vector_field = "vector",
    text_field="text",
    connection_args={"uri": "http://localhost:19530"},
)
retriever = vectorstore.as_retriever()

ConnectionError: HTTPConnectionPool(host='localhost', port=8012): Max retries exceeded with url: /v1/models (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fc0cb784d90>: Failed to establish a new connection: [Errno 111] Connection refused'))

Then, we'll create an RAG chain using [llama-3.1-405b-instruct](https://build.nvidia.com/meta/llama-3_1-405b-instruct) that we can use to query our pdf in natural language

In [3]:
import os 
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# TODO: Add your NVIDIA API key
os.environ["NVIDIA_API_KEY"] = "[YOUR NVIDIA API KEY HERE]"

llm = ChatNVIDIA(model="meta/llama-3.1-405b-instruct")

In [17]:
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Keep the answer concise."
    "\n\n"
    "{context}"
    "Question: {question}"
)

prompt = PromptTemplate.from_template(template)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

And now we can ask our pdf questions

In [16]:
rag_chain.invoke("What is the dog doing and where?")

'The dog is chasing a squirrel in the front yard.'